In [1]:
import pandas as pd

# Loading Data

In [2]:
path = "C:/Users/Daniel/Data Science/Workspace/Instacart Market Basket Analysis/2. Prepared Data/"

In [3]:
# aisles = pd.read_csv(path + "aisles.csv")
# departments = pd.read_csv(path + "departments.csv")

order_products__prior = pd.read_csv(path + "order_products__prior.csv")
order_products__train = pd.read_csv(path + "order_products__train.csv")
orders = pd.read_csv(path + "orders.csv")
products = pd.read_csv(path + "products.csv")

# sample_submission = pd.read_csv(path + "sample_submission.csv")

In [4]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [5]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


# Data Preparation

In [6]:
all_data = orders

order_products = pd.concat([order_products__prior, order_products__train])
all_data = all_data.merge(order_products, on='order_id', how='left')

# products.drop('product_name', axis=1, inplace=True)
# all_data = all_data.merge(products, on='product_id', how='left')

In [7]:
del order_products__prior, order_products__train, order_products #, products

In [8]:
# use 'reordered' as last column
# all_data = all_data[[c for c in all_data if c not in ['reordered']] + ['reordered']]
# all_data.head(10)

In [9]:
prior = all_data[all_data.eval_set=='prior']
train = all_data[all_data.eval_set=='train']
test = all_data[all_data.eval_set=='test']

In [10]:
# remove products that were first ordered in the user's last order (we don't have to predict those)
train = train[train.reordered != 0] 

# column not given for test data
train.drop('add_to_cart_order', inplace=True, axis=1)

train.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered
59,1187899,1,train,11,4,8,14.0,196.0,1.0
60,1187899,1,train,11,4,8,14.0,25133.0,1.0
61,1187899,1,train,11,4,8,14.0,38928.0,1.0
62,1187899,1,train,11,4,8,14.0,26405.0,1.0
63,1187899,1,train,11,4,8,14.0,39657.0,1.0
64,1187899,1,train,11,4,8,14.0,10258.0,1.0
65,1187899,1,train,11,4,8,14.0,13032.0,1.0
66,1187899,1,train,11,4,8,14.0,26088.0,1.0
68,1187899,1,train,11,4,8,14.0,49235.0,1.0
69,1187899,1,train,11,4,8,14.0,46149.0,1.0


# Creating Train Set for Binary Classification
order_products_train indicates which products were reorders in the user's last order <br>
as negative samples we use all products that were previously purchased by the user and not reordered in his last order

In [27]:
# train.groupby('user_id').product_id.apply(set).head()

user_id
1    {196.0, 26405.0, 46149.0, 13032.0, 39657.0, 26...
2    {41787.0, 33957.0, 22825.0, 45066.0, 5450.0, 1...
5                 {21616.0, 40706.0, 15349.0, 21413.0}
7    {17638.0, 29894.0, 47272.0, 45066.0, 13198.0, ...
8                 {15937.0, 41540.0, 23165.0, 21903.0}
Name: product_id, dtype: object

In [12]:
train_orders = orders[orders.eval_set=='train']

In [13]:
reorders = train.groupby('user_id').product_id.apply(set) # reorders in the training order
all_products = prior.groupby('user_id').product_id.apply(set) # all previously purchased products

not_reordered = all_products - reorders

In [14]:
order_list = []
product_list=[]
labels = []

In [15]:
for row in train_orders.itertuples():
    try:
        product_list += reorders[row.user_id]
        order_list += [row.order_id] * len(reorders[row.user_id])
        labels += [1] * len(reorders[row.user_id])
        
        product_list += not_reordered[row.user_id]
        order_list += [row.order_id] * len(not_reordered[row.user_id])
        labels += [0] * len(not_reordered[row.user_id])
    except:
        product_list += all_products[row.user_id]
        order_list += [row.order_id] * len(all_products[row.user_id])
        labels += [0] * len(all_products[row.user_id])

In [16]:
train_df = pd.DataFrame()
train_df['order_id'] = order_list
train_df['product_id'] = product_list
train_df['reordered'] = labels

In [17]:
train_df.head(15)

,order_id,product_id,reordered
0,1187899,196.0,1
1,1187899,26405.0,1
2,1187899,46149.0,1
3,1187899,13032.0,1
4,1187899,39657.0,1
5,1187899,26088.0,1
6,1187899,25133.0,1
7,1187899,38928.0,1
8,1187899,10258.0,1
9,1187899,49235.0,1


In [19]:
train_df = train_df.merge(orders, how='left')
train_df = train_df.merge(products, how='left')

train_df.drop(['eval_set', 'product_name'], axis=1, inplace=True)

train_df = train_df[[column for column in train_df if column != 'reordered'] + ['reordered']]

In [20]:
train_df

,order_id,product_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,reordered
0,1187899,196.0,1,11,4,8,14.0,77,7,1
1,1187899,26405.0,1,11,4,8,14.0,54,17,1
2,1187899,46149.0,1,11,4,8,14.0,77,7,1
3,1187899,13032.0,1,11,4,8,14.0,121,14,1
4,1187899,39657.0,1,11,4,8,14.0,45,19,1
5,1187899,26088.0,1,11,4,8,14.0,23,19,1
6,1187899,25133.0,1,11,4,8,14.0,21,16,1
7,1187899,38928.0,1,11,4,8,14.0,120,16,1
8,1187899,10258.0,1,11,4,8,14.0,117,19,1
9,1187899,49235.0,1,11,4,8,14.0,53,16,1


# Test Set

In [21]:
test_orders = orders[orders.eval_set=='test']

In [22]:
order_list = []
product_list=[]

In [23]:
for row in test_orders.itertuples():
        product_list += all_products[row.user_id]
        order_list += [row.order_id] * len(all_products[row.user_id])

In [24]:
test_df = pd.DataFrame()
test_df['order_id'] = order_list
test_df['product_id'] = product_list

In [25]:
test_df = test_df.merge(orders, how='left')
test_df = test_df.merge(products, how='left')

test_df.drop(['eval_set', 'product_name'], axis=1, inplace=True)

In [26]:
test_df

,order_id,product_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
0,2774568,17668.0,3,13,5,15,11.0,91,16
1,2774568,44683.0,3,13,5,15,11.0,83,4
2,2774568,48523.0,3,13,5,15,11.0,37,1
3,2774568,21903.0,3,13,5,15,11.0,123,4
4,2774568,14992.0,3,13,5,15,11.0,83,4
5,2774568,21137.0,3,13,5,15,11.0,24,4
6,2774568,32402.0,3,13,5,15,11.0,78,19
7,2774568,49683.0,3,13,5,15,11.0,83,4
8,2774568,22035.0,3,13,5,15,11.0,21,16
9,2774568,39190.0,3,13,5,15,11.0,91,16
